# crawl more details
Please run sina_sales.ipynb first!

In [1]:
# Global Constants
import os
import json
import pandas as pd
import numpy as np

data_directory='./data_all'
data_directory_e='./data_e'
data_directory_eInfo_1='./data_eInfo_1'
data_directory_carIds='./data_carIds'
data_directory_detail='./data_detail'

csv_path='df_sales_all.csv'
csv_e_path='df_sales_e.csv'
csv_all_e_path='df_sales_all_e.csv'
csv_detail='df_detail.csv'

if not os.path.exists(data_directory_e):
    os.makedirs(data_directory_e)
if not os.path.exists(data_directory_eInfo_1):
    os.makedirs(data_directory_eInfo_1)
if not os.path.exists(data_directory_carIds):
    os.makedirs(data_directory_carIds)
if not os.path.exists(data_directory_detail):
    os.makedirs(data_directory_detail)

## crawl new energy cars
crawl latest month sales data

In [2]:
# Constants
carGuidePrice="" # example "10-15" 
corpType=""
energyType="3,2,1-6"
base="https://price.auto.sina.com.cn/api/PaihangbangSales/getSubSalesByParams?size=30"

def generateCommand(year="2023",month="01",page="1"):
    url=f'\"{base}&page={page}&year={year}&month={month}&carGuidePrice={carGuidePrice}&serialJiBie=&corpType={corpType}&ranliaoXingshi={energyType}&serial_id=&need_detail=1\"'

    fileName=f'{year}-{month}_{page}.json'
    command=f'curl {url} -o {data_directory_e}/{fileName}'
    return command

In [3]:
year='2023'
month='05'
script=f'{data_directory_e}/getJSON-{year}-{month}.ps1'
print(script)
with open(script,"w") as f:
    for i in range(30):
        print(generateCommand(year,month,page=i),file=f)

./data_e/getJSON-2023-05.ps1


In [4]:
if 0:# /1 to scrawl
    !powershell -ExecutionPolicy Bypass -File $script

In [5]:

def extractData(filepath,df):
    with open(filepath,'r',encoding="utf8") as file:
        file_data=json.load(file)
        date = filepath.split('/')[-1].split('.')[0].split('_')[0]

        for i in range(len(file_data['data']['list'])):
            sub_brand_id=int(file_data['data']['list'][i]['sub_brand_id'])

            sales_volume=file_data['data']['list'][i]['sales_volume']
            
            if  not sub_brand_id in df.index.values:
                serialZhName=file_data['data']['list'][i]['serial_info']['serialZhName']
                df.loc[sub_brand_id, 'serialZhName']=serialZhName
                df.loc[sub_brand_id, 'url']=f'https://db.auto.sina.com.cn/{sub_brand_id}'
            df.loc[sub_brand_id, date] = sales_volume
                

In [6]:

df_sales_e = pd.DataFrame(index=pd.Index([], name='sub_brand_id'))
# df_info= pd.DataFrame(index=pd.Index([], name='sub_brand_id'), columns=['info'])
for filename in os.listdir(data_directory_e):
    if filename.endswith('.json'):
        filepath = f'{data_directory_e}/{filename}'
        extractData(filepath,df_sales_e)
df_sales_e=df_sales_e.fillna(0)

crawl more infomation from `https://db.auto.sina.cn/inter/serial/someinfo/`

In [7]:

base="https://db.auto.sina.cn/inter/serial/someinfo/"

def generateCommand_eInfo_1(sub_brand_id=4281):
    url=f'\"{base}/{sub_brand_id}/\"'

    fileName=f'info_{sub_brand_id}.json'
    command=f'curl {url} -o {data_directory_eInfo_1}/{fileName}'
    return command

In [8]:
script=f'{data_directory_eInfo_1}/getJSON-eInfo_1.ps1'
with open(script,"w") as f:
    for sub_brand_id in df_sales_e.index:
        print(generateCommand_eInfo_1(sub_brand_id),file=f)

In [9]:
if 0:# /1 to scrawl
    !powershell -ExecutionPolicy Bypass -File $script

In [10]:
def extractData_eInfo(filepath,df):
    with open(filepath,'r',encoding="utf8") as file:
        file_data=json.load(file)
        sub_brand_id=int(file_data['data']["serialId"])
        
        df.loc[sub_brand_id, "rechargeMileage/km"] = file_data['data']["xuhanglichengMax"]
        df.loc[sub_brand_id, "minimiumChargeTime/h"] = file_data['data']["kuaichongshijianMin"]


In [11]:
for filename in os.listdir(data_directory_eInfo_1):
    if filename.endswith('.json'):
        filepath = f'{data_directory_eInfo_1}/{filename}'
        extractData_eInfo(filepath,df_sales_e)
df_sales_e

,serialZhName,url,2023-04,2023-05,rechargeMileage/km,minimiumChargeTime/h
sub_brand_id,,,,,,
4261,秦PLUS DM-i,https://db.auto.sina.com.cn/4261,30366,28093,120,
4281,Model Y,https://db.auto.sina.com.cn/4281,27276,31980,660,1.0
4411,海豚,https://db.auto.sina.com.cn/4411,22709,29593,420,0.5
4507,元PLUS,https://db.auto.sina.com.cn/4507,21387,23937,510,0.5
86,雅阁,https://db.auto.sina.com.cn/86,19636,12971,,
...,...,...,...,...,...,...
3906,蔚来EC6,https://db.auto.sina.com.cn/3906,0,28,,
2971,沃尔沃S60混动,https://db.auto.sina.com.cn/2971,0,27,84,
4186,荣威ei6 MAX,https://db.auto.sina.com.cn/4186,0,26,70,


In [12]:
df_sales_e.to_csv(csv_e_path)

In [13]:
df_sales_e=pd.read_csv(csv_e_path)
df_sales_e.set_index('sub_brand_id', inplace = True)
df_sales_e

,serialZhName,url,2023-04,2023-05,rechargeMileage/km,minimiumChargeTime/h
sub_brand_id,,,,,,
4261,秦PLUS DM-i,https://db.auto.sina.com.cn/4261,30366,28093,120.0,NaN
4281,Model Y,https://db.auto.sina.com.cn/4281,27276,31980,660.0,1.00
4411,海豚,https://db.auto.sina.com.cn/4411,22709,29593,420.0,0.50
4507,元PLUS,https://db.auto.sina.com.cn/4507,21387,23937,510.0,0.50
86,雅阁,https://db.auto.sina.com.cn/86,19636,12971,NaN,NaN
...,...,...,...,...,...,...
3906,蔚来EC6,https://db.auto.sina.com.cn/3906,0,28,NaN,NaN
2971,沃尔沃S60混动,https://db.auto.sina.com.cn/2971,0,27,84.0,NaN
4186,荣威ei6 MAX,https://db.auto.sina.com.cn/4186,0,26,70.0,NaN


---

Get carIds from `https://dbs.auto.sina.cn/api/auto/getCarIdBySearch?serialId=<sub_brand_id>`

In [14]:
base="https://dbs.auto.sina.cn/api/auto/getCarIdBySearch?serialId="

def generateCommand_carIds(sub_brand_id=4281):
    url=f'\"{base}{sub_brand_id}\"'

    fileName=f'carIds_{sub_brand_id}.json'
    command=f'curl {url} -o {data_directory_carIds}/{fileName}'
    return command

In [15]:
df_sales_all_e=pd.read_csv(csv_path)
df_sales_all_e.set_index('sub_brand_id', inplace = True)
# df_sales_all_e

In [16]:
script=f'{data_directory_carIds}/getJSON-carIds.ps1'
with open(script,"w") as f:
    for sub_brand_id in df_sales_all_e.index:
        print(generateCommand_carIds(sub_brand_id),file=f)

In [17]:
## Regenerate unsuccessful requests
successJson=os.listdir(data_directory_carIds)
script=f'{data_directory_carIds}/reGetJSON.ps1'
with open(script,"w") as f:
    for sub_brand_id in df_sales_all_e.index:
        fileName=f'carIds_{sub_brand_id}.json'
        if not fileName in successJson:
            print(generateCommand_carIds(sub_brand_id),file=f)   

In [18]:
script

'./data_carIds/reGetJSON.ps1'

In [19]:
if 0:# /1 to scrawl
    !powershell -ExecutionPolicy Bypass -File $script

In [20]:
def extractParams(file_data,param):
    content=file_data['data']["params"][param]
    if isinstance(content,dict):
        arr=list(content.values())
    elif isinstance(content,list):
        arr=content
    return ', '.join(map(str, arr))
    
def extractData_carIds(filepath,df):
    with open(filepath,'r',encoding="utf8") as file:
        file_data=json.load(file)
        sub_brand_id=int(file_data['data']["serialInfo"]["serialId"])

        
        df.loc[sub_brand_id, "brandId"] = file_data['data']["serialInfo"]['brandId']
        df.loc[sub_brand_id, "brandZhName"] = file_data['data']["serialInfo"]["brandZhName"]
        df.loc[sub_brand_id, "corpId"] = file_data['data']["serialInfo"]['corpId']
        df.loc[sub_brand_id, "corpName"] = file_data['data']["serialInfo"]["corpName"]
        df.loc[sub_brand_id, "serialGuidePriceArea"] = file_data['data']["serialInfo"]["serialGuidePriceArea"]
        df.loc[sub_brand_id, "hasNewEnergy"] = file_data['data']["serialInfo"]["hasNewEnergy"]
   
            
        df.loc[sub_brand_id, "fueltypeList"] = extractParams(file_data,"fueltypeList")
        df.loc[sub_brand_id, "enginetypeList"] = extractParams(file_data,"enginetypeList")
        df.loc[sub_brand_id, "drivetypeList"] = extractParams(file_data,"drivetypeList")
        df.loc[sub_brand_id, "mileageList"] = extractParams(file_data,"mileageList")
        df.loc[sub_brand_id, "cellcapList"] = extractParams(file_data,"cellcapList")
        df.loc[sub_brand_id, "drivetypeList"] = extractParams(file_data,"drivetypeList")
        df.loc[sub_brand_id, "yearList"] = extractParams(file_data,"yearList")
        
        df.loc[sub_brand_id, "carIds"] = ','.join(map(str, file_data['data']['carIds']))

In [21]:
i=0
for filename in os.listdir(data_directory_carIds):
    if filename.endswith('.json'):
        filepath = f'{data_directory_carIds}/{filename}'
        print(f"\r>> You reached file {i},filepath {filepath}", end='')
        i+=1
        extractData_carIds(filepath,df_sales_all_e)
        
# df_sales_all_e

>> You reached file 922,filepath ./data_carIds/carIds_982.jsonn

In [22]:
df_sales_all_e.to_csv(csv_all_e_path)

crawl details in `https://dbs.auto.sina.cn/api/auto/getGroupCarInfo?carIds=<carIds>`

In [23]:
df_sales_all_e=pd.read_csv(csv_all_e_path)
df_sales_all_e.set_index('sub_brand_id', inplace = True)
df_sales_all_e

,serialZhName,url,2020-01,2020-02,2020-03,2020-04,2020-05,2020-06,2020-07,2020-08,...,corpName,serialGuidePriceArea,hasNewEnergy,fueltypeList,enginetypeList,drivetypeList,mileageList,cellcapList,yearList,carIds
sub_brand_id,,,,,,,,,,,,,,,,,,,,,
428,轩逸,https://db.auto.sina.com.cn/428,50275,4463,24109,32063,43922,47654,47123,47281,...,东风日产,10.86-14.08万元,0,汽油,1.6L,前驱,NaN,NaN,"2023, 2022, 2021, 2020, 2019, 2018, 2016, 2014...","63585,62450,62451,54400,54401,58822"
549,朗逸,https://db.auto.sina.com.cn/549,45836,3547,21713,26712,31294,29592,32845,40498,...,上汽大众,9.40-15.19万元,0,汽油,"1.2T, 1.4T, 1.5L, 1.5T",前驱,NaN,NaN,"2023, 2022, 2021, 2019, 2018, 2017, 2016, 2015...","65657,65658,65659,64761,64762,64765,64764,6476..."
776,哈弗H6,https://db.auto.sina.com.cn/776,37674,3573,15036,17087,19109,20315,23742,24518,...,长城汽车,9.89-15.70万元,0,"汽油, 油电混合","1.5T, 2.0T","前驱, 四驱",NaN,NaN,"2023, 2022, 2021, 2020, 2019, 2018, 2017, 2016...","64800,64799,61067,61068,61069,61070,60153,4783..."
492,卡罗拉,https://db.auto.sina.com.cn/492,31392,4206,19396,28406,32964,33546,31485,30342,...,一汽丰田,10.98-15.98万元,0,"汽油, 油电混合","1.2T, 1.5L, 1.8L",前驱,NaN,NaN,"2023, 2022, 2021, 2019, 2018, 2017, 2016, 2014...","65271,65275,65272,65273,65276,65277,65274,6527..."
573,宝来,https://db.auto.sina.com.cn/573,30557,2667,17975,21357,23533,23023,25102,29643,...,一汽-大众,9.88-15.70万元,0,汽油,"1.2T, 1.4T, 1.5L, 1.5T",前驱,NaN,NaN,"2023, 2021, 2020, 2019, 2018, 2017, 2016, 2015...","58964,61508,58965,60633,58966,60634,58967,5896..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5351,家宝,https://db.auto.sina.com.cn/5351,0,0,0,0,0,0,0,0,...,北汽制造,3.98-5.69万元,1,纯电动,NaN,后驱,"122, 172, 205","10.88, 15.12, 17.28",2023,"65502,62793,65503,65504,62794,62795"
5115,奔腾M9,https://db.auto.sina.com.cn/5115,0,0,0,0,0,0,0,0,...,一汽奔腾,22.98-24.98万元,0,汽油+48V轻混系统,2.0T,前驱,NaN,NaN,2023,"63716,63717"
4595,海鸥,https://db.auto.sina.com.cn/4595,0,0,0,0,0,0,0,0,...,比亚迪,7.38-8.98万元,1,纯电动,NaN,前驱,"305, 405","30.08, 38.88",2023,"64551,64552,64553"


In [24]:
base="https://dbs.auto.sina.cn/api/auto/getGroupCarInfo?carIds="

def generateCommand_detail(sub_brand_id=4281):

    fileName=f'detail_{sub_brand_id}.json'
    carIds=df_sales_all_e['carIds'][sub_brand_id]
    url=f'\"{base}{carIds}\"'
    command=f'curl {url} -o {data_directory_detail}/{fileName}'
    return command

In [25]:
script=f'{data_directory_detail}/getJSON-detail.ps1'
with open(script,"w") as f:
    for sub_brand_id in df_sales_all_e.index:
        print(generateCommand_detail(sub_brand_id),file=f)

In [26]:
## Regenerate unsuccessful requests
successJson=os.listdir(data_directory_detail)
script=f'{data_directory_detail}/reGetJSON.ps1'
with open(script,"w") as f:
    for sub_brand_id in df_sales_all_e.index:
        fileName=f'detail_{sub_brand_id}.json'
        if not fileName in successJson:
            print(generateCommand_detail(sub_brand_id),file=f)   

In [27]:
script

'./data_detail/reGetJSON.ps1'

In [28]:
if 0:# /1 to scrawl
    !powershell -ExecutionPolicy Bypass -File $script

In [29]:
def extractData_carIds(filepath,df):
    with open(filepath,'r',encoding="utf8") as file:
        file_data=json.load(file)
        carIds=file_data['data']["allCar"]
        for dataItem in file_data['data']['data']:
            if dataItem['title'] in ["基本信息","车身","电动机","发动机","底盘/转向","车轮/制动"]:              
                for item in dataItem['list']:
                    content=item['data']
                    for carid in list(content.keys()):
                        carid=int(carid)
                        name=f"{item['name']}/{item['en_name']}"
                        if carid in content:
                            df.loc[carid,name] = content[carid]
                        else:
                            pass
                            df.loc[carid,name] = content[str(carid)]

In [ ]:
df_detail = pd.DataFrame(columns=['carIds'])
df_detail.set_index('carIds',inplace = True)
i=0
for filename in os.listdir(data_directory_detail):
    if filename.endswith('.json'):
        filepath = f'{data_directory_detail}/{filename}'
        print(f"\r>> You reached file {i},filepath {filepath}", end='')
        i+=1
        extractData_carIds(filepath,df_detail)         
#         break
df_detail

>> You reached file 754,filepath ./data_detail/detail_4652.json

In [ ]:
df_detail.to_csv(csv_detail)